In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import cvxpy as cp

In [2]:
# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

[-1.79109255e-19  2.85112420e-02  2.79973443e-19  3.37658729e-20
 -2.72802663e-19  1.49285011e-01 -9.94082533e-20  8.35373900e-20
  2.46718649e-01  5.78224144e-01 -4.03739463e-19  1.01242860e-03
 -9.28486180e-20  2.26767464e-01 -1.58813678e-19 -8.97232272e-20
 -1.22145729e-19 -1.51509428e-19  1.12060672e-19 -3.48318635e-19]
[ 2.50938945  0.          2.78354615  1.79425782 13.08579183  0.
  0.73716363  3.35344995  0.          0.          8.93825054  0.
  7.02955161  0.          4.71068649  3.18873635  2.06090107 10.08166738
  3.0481157   8.53268239]


In [7]:
np.hstack([np.array([[3,3],[4,4]]), np.ones(2).reshape(-1, 1)])

array([[3., 3., 1.],
       [4., 4., 1.]])

In [115]:
def softmax(M, axis=-1):
    return cp.exp(M - cp.log(cp.sum(cp.exp(M), axis=axis, keepdims=True)))

def strategic_svm(X, A, u, C=1):
    n, d = X.shape
    _, m = A.shape
    X = np.hstack([X, np.ones(n).reshape(-1, 1)]) # attach bias column
    one_m = np.ones(m)
    phi = cp.Variable([m, d+1])
    zeta = cp.Variable(n)
    
    objective = cp.Minimize(cp.sum_squares(phi) + C * cp.sum(zeta))
    constraints = [cp.multiply((u @ A.T), cp.sum(cp.multiply(A, (phi @ X.T).T), axis=1)) >= 1 - zeta,
                  phi >= 0]
    prob = cp.Problem(objective, constraints)#, constraints)
    result = prob.solve()#qcp=True)
    
    return phi.value

In [143]:
def simple_strategic_svm(X, Y, reward=0.3, C=1):
    n, d = X.shape
    X = np.hstack([X, np.ones(n).reshape(-1, 1)]) # attach bias column
    phi = cp.Variable(d+1)
    zeta = cp.Variable(n)
    
    objective = cp.Minimize(cp.sum_squares(phi) + C * cp.sum(zeta))
    constraints = [cp.multiply(reward * Y, cp.multiply(Y, (phi @ X.T).T)) >= 1 - zeta,
                  phi >= 0]
    prob = cp.Problem(objective, constraints)#, constraints)
    result = prob.solve()#qcp=True)
    
    return phi.value

In [144]:
phi = np.random.randn(3)
X = np.array([[1,1,1],[3,3,1],[5,5,1],[7,7,1]])
X_oneless = X[:,:-1]
Y = np.array([1, 1, -1, -1])
simple_strategic_svm(X_oneless, Y, reward=0.3)

array([2.4, 2.4, 0.6])

In [139]:
def normal_svm(X, A, u, C=1):
    n, d = X.shape
    _, m = A.shape
    X = np.hstack([X, np.ones(n).reshape(-1, 1)]) # attach bias column
    one_m = np.ones(m)
    phi = cp.Variable([m, d+1])
    zeta = cp.Variable(n)
    
    objective = cp.Minimize(cp.sum_squares(phi) + C * cp.sum(zeta))
    constraints = [cp.sum(cp.multiply(A, (phi @ X.T).T), axis=1) >= 
                   1 - zeta,
                  phi >= 0]
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    return phi.value

def normal_svm(X, A, u, C=1):
    n, d = X.shape
    _, m = A.shape
    X = np.hstack([X, np.ones(n).reshape(-1, 1)]) # attach bias column
    one_m = np.ones(m)
    phi = cp.Variable([m, d+1])
    zeta = cp.Variable(n)
    
    objective = cp.Minimize(cp.sum_squares(phi) + C * cp.sum(zeta))
    constraints = [cp.norm2((X @ phi.T) - A, axis=1) >= 1 - zeta,
                  phi >= 0]
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    return phi.value

In [140]:
phi = normal_svm(X_oneless, A, u)

TypeError: sum_squares() got an unexpected keyword argument 'dim'

In [124]:
phi

array([[18., 18.,  3.],
       [ 4.,  4.,  2.]])

In [38]:
np.hstack([X, np.ones(4).reshape(-1, 1)])
X.shape

(4, 3)

In [11]:
phi = np.random.randn(2, 3)
X = np.array([[1,1,1],[3,3,1],[5,5,1],[7,7,1]])
X_oneless = X[:,:-1]
X_oneless
A = np.array([[0,1], [0,1], [1,0], [1,0]])
one_m = np.ones(2)
phi

array([[-0.59334375,  0.5103076 , -0.91579185],
       [-0.40725204,  0.98495167,  1.07125243]])

In [30]:
u = np.array([3,2])# .reshape(1, -1)
u

array([3, 2])

In [61]:
np.multiply(A, (phi @ X.T).T).shape

(4, 2)

In [19]:
np.sum(A * (phi @ X.T).T, axis=-1)

array([ 1.64895206,  2.80435131, -1.33097264, -1.49704495])

In [31]:
u @ (phi @ X.T)

array([0.30142009, 2.11400166, 3.92658323, 5.7391648 ])

In [50]:
one_m @ (phi @ X.T)

array([0.65012405, 1.63945099, 2.62877793, 3.61810488])